In [ ]:
# IMPORTING LIBRARIES
import copy
import os
import random
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'  # or ':16:8'
import numpy as np
import torch
from torch_geometric.nn import global_mean_pool
from torch_geometric.loader import DataLoader
# FUNCTIONS
from data_processing import load_dataset, smiles_to_graph, process_dataset
from path_helpers import get_path
from stats_compute import compute_statistics, scale_graphs
from smart_loader import load_model_for_inference
from EnhancedDataSplit import DataSplitter
# DIRECTORY SETUP
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)

In [ ]:
# HYPERPARAMETER SETTINGS
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# Reproducibility settings
seed = 21
split_seed = 42
# Hyperparameters
batch_size = 32
runtime = timestamp

###########################
model_inference_dir = os.path.join(
    os.path.dirname(os.getcwd()),
    "models",
    "models_root",
    "model_for_inference",
    "ras_pinn"
)
# List files and pick the first one
files = sorted(os.listdir(model_inference_dir))  # sorted to make it deterministic
if len(files) == 0:
    raise FileNotFoundError(f"No files found in {model_inference_dir}")
model_file = files[0]  # first file in directory
print(f"Using model file: {model_file}")
# Full path
path = os.path.join(model_inference_dir, model_file)
########### IMPORTING MODEL ###############
selected_device = 'cuda'
device = torch.device(selected_device)
model = load_model_for_inference(path, device=device)
############################################
###########################

# CUDA Deterministic (ON/OFF SETTING)
# For PyTorch
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.use_deterministic_algorithms(False)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True


print('device           :', device)
print('seed             :', seed)
print('split seed       :', split_seed)

In [ ]:
# LOAD & GRAPH GENERATION FOR EITHER SRS OR RAS
df_components = load_dataset(get_path(file_name = 'components_set.csv', folder_name='datasets'))
smiles_dict = dict(zip(df_components['Abbreviation'], df_components['SMILES']))
df_systems = load_dataset(get_path(file_name = 'systems_set.csv', folder_name='datasets'))
smiles_list = df_components["SMILES"].dropna().tolist()
mol_name_dict = smiles_dict.copy()
# GRAPH
system_graphs = process_dataset(df_systems, smiles_dict)
# LOAD DATASET
splitter = DataSplitter(system_graphs, random_state=split_seed)
splitter.print_dataset_stats()
# Options: rarity_aware_unseen_amine_split stratified_random_split
train_data, val_data, test_data = splitter.rarity_aware_unseen_amine_split()
#Retrieve the statistics of train_data
stats = compute_statistics(train_data)
conc_mean = stats[0]
conc_std = stats[1]
temp_mean = stats[2]
temp_std = stats[3]
pco2_mean = stats[4]
pco2_std = stats[5]
#Apply the scaling to validation and test
original_train_data = copy.deepcopy(train_data)
original_val_data = copy.deepcopy(val_data)
original_test_data = copy.deepcopy(test_data)
combined_original_data = original_train_data + original_val_data + original_test_data
train_data = scale_graphs(train_data, conc_mean, conc_std, temp_mean, temp_std, pco2_mean, pco2_std)
val_data = scale_graphs(val_data, conc_mean, conc_std, temp_mean, temp_std, pco2_mean, pco2_std)
test_data = scale_graphs(test_data, conc_mean, conc_std, temp_mean, temp_std, pco2_mean, pco2_std)
#Load the data into DataLoader
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
# Mol name processing
test_molecules = list(set([data.name for data in test_data]))
# Separate by molecule name
molecule_groups_test = {}
for mol_name in test_molecules:
   molecule_groups_test[mol_name] = [data for data in test_data if data.name == mol_name]

val_molecules = list(set([data.name for data in val_data]))

# Separate by molecule name
molecule_groups_val = {}
for mol_name in val_molecules:
   molecule_groups_val[mol_name] = [data for data in val_data if data.name == mol_name]

In [ ]:
# MODEL WRAPPER
class EmbeddingsExtractor(torch.nn.Module):
    def __init__(self, full_model):
        super().__init__()
        self.graph_block = full_model.graph_block
        self.use_adaptive_pooling = full_model.use_adaptive_pooling
        if self.use_adaptive_pooling:
            self.adaptive_pool = full_model.adaptive_pool

    def forward(self, data, extract_embeddings=False, include_conditions=False):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        x = self.graph_block(x, edge_index, edge_attr)
        
        if self.use_adaptive_pooling:
            x = self.adaptive_pool(x, batch)
        else:
            x = global_mean_pool(x, batch)

        if extract_embeddings:
            return x

        if include_conditions:
            additional_features = torch.stack([data.conc, data.temp, data.pco2], dim=1).float()
            x = torch.cat([x, additional_features], dim=1)
        return x

class FCNNWrapper:
    def __init__(self, fc_block, device):
        self.fc_block = fc_block
        self.device = device
        self.fc_block.eval()
    def __call__(self, x_numpy):
        x_tensor = torch.from_numpy(x_numpy).float().to(self.device)
        with torch.no_grad():
            y = self.fc_block(x_tensor)
        return y.cpu().numpy()
fc_wrapper = FCNNWrapper(model.fc_block, device)

In [ ]:
# Filter molecular graph from combined dataset  & define embedding_generator
unique_named_graphs = {}
for graph in combined_original_data:
    name = graph['name']
    if name not in unique_named_graphs:
        unique_named_graphs[name] = graph
unique_graph_list = list(unique_named_graphs.values())
mol_loader = DataLoader(unique_graph_list, batch_size=batch_size, shuffle=False)
# Execute the EmbeddingsExtractor
embedding_generator = EmbeddingsExtractor(model).to(device)
embedding_generator.eval()
all_embeddings = []
all_amines = []
all_data = []
with torch.no_grad():
    for data in mol_loader:
        data = data.to(device)
        combined_embedding = embedding_generator(data, 
                                                 include_conditions=False, 
                                                 extract_embeddings=True)
        all_embeddings.append(combined_embedding.cpu())
        all_amines.extend(data.name)
        all_data.extend(data.cpu().to_data_list())

# Now after the loop
all_embeddings = torch.cat(all_embeddings)
index_to_name = {i: name for i, name in enumerate(all_amines)}

# Assigning extracted embeddings to Data object as tensor
name_to_embedding = {name: emb for name, emb in zip(all_amines, all_embeddings)}
for dataset in [train_data, val_data, test_data]:
    for data in dataset:
        name = data.name
        data.embedding = name_to_embedding[name]
        if name in smiles_dict:
            data.smiles = smiles_dict[name]
        else:
            data.smiles = None

def extract_combined_vectors(loader, model, embedding_generator, device):
    vectors = []
    model.eval()
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            graph_emb = embedding_generator(data, extract_embeddings=True)
            ext_feats = torch.stack([data.conc, data.temp, data.pco2], dim=1).float()
            combined = torch.cat([graph_emb, ext_feats], dim=1)
            vectors.append(combined.cpu())
    return torch.cat(vectors).numpy()
background_vectors = extract_combined_vectors(train_loader, model, embedding_generator, device)
val_vectors        = extract_combined_vectors(val_loader, model, embedding_generator, device)
test_vectors       = extract_combined_vectors(test_loader, model, embedding_generator, device)
num_emb_dims = test_vectors.shape[1] - 3
feature_names = [f"embedding_{i}" for i in range(num_emb_dims)] + ['conc', 'temp', 'pco2']

combined_data_with_embeddings = train_data + val_data + test_data
unique_named_graphs_with_embeddings = {}
for graph in combined_data_with_embeddings:
    name = graph['name']
    if name not in unique_named_graphs_with_embeddings:
        unique_named_graphs_with_embeddings[name] = graph
unique_graph_list_with_embeddings = list(unique_named_graphs_with_embeddings.values())

In [ ]:
""" from collections import defaultdict
import numpy as np
import warnings
import shap

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message=".*NumPy global RNG.*")

# ---- 1) Prepare background once ----
background = np.stack([
    np.concatenate([data.embedding.numpy(), 
                    np.array([data.conc, data.temp, data.pco2])])
    for data in train_data
])
background = background[np.random.choice(background.shape[0], 100, replace=False)]

explainer = shap.KernelExplainer(fc_wrapper, background)

# ---- 2) Prepare full test set ----
test_vectors = np.stack([
    np.concatenate([data.embedding.numpy(), 
                    np.array([data.conc, data.temp, data.pco2])])
    for data in val_data
])
molecule_names = [data.name for data in val_data]

# ---- 3) Compute SHAP for full test set ----
shap_values = explainer.shap_values(test_vectors)

# ---- 4) Plot summary for all test samples ----
shap.summary_plot(
    np.squeeze(shap_values),
    test_vectors,
    feature_names=feature_names,
    plot_type="dot",
    show=True,
    max_display=10
)

# ---- 5) Extract top 5 embeddings per molecule ----
embedding_size = len(train_data[0].embedding.numpy())
molecule_top_embeddings = defaultdict(dict)

for molecule_name in set(molecule_names):
    indices_mask = [i for i, name in enumerate(molecule_names) if name == molecule_name]
    molecule_vectors = test_vectors[indices_mask]
    molecule_shap = np.abs(np.squeeze(shap_values)[indices_mask, :embedding_size]).mean(axis=0)
    top5_indices = np.argsort(molecule_shap)[-5:][::-1]
    avg_embeddings = molecule_vectors[:, :embedding_size].mean(axis=0)
    top5_embeddings = avg_embeddings[top5_indices]
    
    molecule_top_embeddings[molecule_name] = {
        'indices': top5_indices,
        'values': top5_embeddings,
        'importance_scores': molecule_shap[top5_indices]
    }

    print(f"\n=== {molecule_name} ===")
    print(f"Top 5 embedding indices: {top5_indices}")
    print(f"Top 5 embedding values: {top5_embeddings}")
    print(f"Importance scores: {molecule_shap[top5_indices]}") """

In [ ]:
# Fine grained prediction
from torch_geometric.data import Batch
import torch

def predict_from_smiles(model, smiles_list, conc_list, temp_list, pco2_list,
                        conc_mean, conc_std, temp_mean, temp_std, pco2_mean, pco2_std,
                        device='cuda'):
    """
    Predict alpha_CO2 from raw SMILES + concentration/temp/pco2,
    using scaling from training set. Supports CUDA.
    """
    model.eval()
    model.to(device)
    data_list = []

    for smi, conc, temp, pco2 in zip(smiles_list, conc_list, temp_list, pco2_list):
        # Convert SMILES → PyG Data
        data = smiles_to_graph(smi, None)

        # Apply scaling
        conc_scaled = (conc - conc_mean) / conc_std
        temp_scaled = (temp - temp_mean) / temp_std
        pco2_scaled = (pco2 - pco2_mean) / pco2_std

        # Attach features as tensors on the same device
        data.conc = torch.tensor(conc_scaled, dtype=torch.float, device=device)
        data.temp = torch.tensor(temp_scaled, dtype=torch.float, device=device)
        data.pco2 = torch.tensor(pco2_scaled, dtype=torch.float, device=device)

        data_list.append(data)

    # Batch all molecules and move to device
    batch_data = Batch.from_data_list(data_list).to(device)

    # Forward pass
    with torch.no_grad():
        preds = model(batch_data).squeeze()

    return preds.cpu().numpy()
def round_significant(x, sig=3):
    """
    Round array or number x to `sig` significant digits
    """
    x = np.array(x)
    return np.array([float(f"{v:.{sig}g}") for v in x])

In [ ]:
unique_graph_list_with_embeddings
for i, j in enumerate(unique_graph_list_with_embeddings):
    print(f"{i}: {j['name']} - {j['smiles']} - {j['type']}")

In [ ]:
# Prediction maker (DGA as baseline)
smiles_list = ['C(COCCO)N' , 'C(COCC)N', 'C(COCCN)N','C(CCCO)N']
conc = 2.5
temp = 313.15
pco2 = 15

# Repeat the scalars for each SMILES
conc_list = [conc] * len(smiles_list)
temp_list = [temp] * len(smiles_list)
pco2_list = [pco2] * len(smiles_list)

preds = round_significant(
    predict_from_smiles(
        model, smiles_list, conc_list, temp_list, pco2_list,
        conc_mean, conc_std, temp_mean, temp_std, pco2_mean, pco2_std,
        device='cuda'
    ),
    sig=5
)
for smi, pred in zip(smiles_list, preds):
    print(40 * '-')
    print(f"SMILES -> {smi} \n Conc: {conc} M \n Temp: {temp} K \n pCO2: {pco2} kPa \n α Pred -> {pred} mol/mol")
    print(40 * '-')

In [ ]:
# DGA stress-test
import numpy as np
import matplotlib.pyplot as plt
fontsize = 16
# Set Times New Roman font for all text
plt.rcParams['font.family'] = 'Times New Roman'
# Define the molecules and conditions
smiles_list = ['C(COCCO)N' , 'C(COCCO)','C(COCC)N','C(COCCN)N', 'C(COCCO)O']
molecule_names = ['DGA', 'Remove -N', 'Remove -OH', 'Replace -OH', 'Replace -N']
conc = 2.5
temp = 313.15

# Generate pCO2 range (1 to 1000 kPa, log scale)
pco2_range = np.logspace(0, 3, 20)  # 20 points from 10^0 to 10^3

# Store results
isotherms = {}

for i, smiles in enumerate(smiles_list):
    print(f"Processing {molecule_names[i]} ({smiles})...")
    
    # Create lists for this molecule across all pCO2 values
    smiles_batch = [smiles] * len(pco2_range)
    conc_batch = [conc] * len(pco2_range)
    temp_batch = [temp] * len(pco2_range)
    pco2_batch = pco2_range.tolist()
    
    # Get predictions
    preds = predict_from_smiles(
        model, smiles_batch, conc_batch, temp_batch, pco2_batch,
        conc_mean, conc_std, temp_mean, temp_std, pco2_mean, pco2_std,
        device='cuda'
    )
    
    isotherms[molecule_names[i]] = {
        'pco2': pco2_range,
        'alpha': preds,
        'smiles': smiles
    }

# Plot the isotherms
plt.figure(figsize=(6, 5))
colors = ['blue', 'red', 'green', 'orange', 'black']
linestyles = ['-', '--', '-', ':', '-']

for i, (name, data) in enumerate(isotherms.items()):
    plt.semilogx(data['pco2'], data['alpha'], 
                 color=colors[i], linestyle=linestyles[i], 
                 linewidth=2, markersize=4,
                 label=f"{name} : {data['smiles']}")

plt.xlabel('CO$_{2}$ Partial Pressure (kPa)', fontsize=fontsize)
plt.ylabel(r'CO$_{\mathrm{2}}$ Loading (mol$_{\mathrm{CO2}}$/mol$_{\mathrm{A}}$)', fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.xlim(1, 1000)
#plt.ylim(0, 1.5)
plt.xscale('log')
from matplotlib.ticker import MultipleLocator
plt.gca().yaxis.set_major_locator(MultipleLocator(0.2))
plt.gca().yaxis.set_minor_locator(MultipleLocator(0.25))
plt.minorticks_on()

plt.grid(True, which='both', linewidth=0.5)
plt.legend(
    bbox_to_anchor=(0, 1.02, 1, 0.2),
    loc='lower left',
    mode='expand',
    frameon=False,
    fontsize=fontsize-7,
    ncol=2,
    markerscale=1,
    labelspacing=1,
    borderpad=0.5,
    columnspacing=0.5,
)

plt.tight_layout()
plt.show()

# Print some key values for comparison
print("\nComparison at pCO2 = 15 kPa:")
for name, data in isotherms.items():
    # Find closest value to 15 kPa
    idx = np.argmin(np.abs(data['pco2'] - 15))
    print(f"{name}: α = {data['alpha'][idx]:.3f} mol/mol")

In [ ]:
# AEEA stress-test
import numpy as np
import matplotlib.pyplot as plt
fontsize = 16
# Set Times New Roman font for all text
plt.rcParams['font.family'] = 'Times New Roman'
# Define the molecules and conditions
smiles_list = ['C(CNCCO)N' , 'C(CCNCO)N','C(NCCCO)N','C(COCCN)N']
molecule_names = ['M0', 'M1', 'M2', 'M3']
conc = 2.5
temp = 313.15

# Generate pCO2 range (1 to 1000 kPa, log scale)
pco2_range = np.logspace(0, 3, 20)  # 20 points from 10^0 to 10^3

# Store results
isotherms = {}

for i, smiles in enumerate(smiles_list):
    print(f"Processing {molecule_names[i]} ({smiles})...")
    
    # Create lists for this molecule across all pCO2 values
    smiles_batch = [smiles] * len(pco2_range)
    conc_batch = [conc] * len(pco2_range)
    temp_batch = [temp] * len(pco2_range)
    pco2_batch = pco2_range.tolist()
    
    # Get predictions
    preds = predict_from_smiles(
        model, smiles_batch, conc_batch, temp_batch, pco2_batch,
        conc_mean, conc_std, temp_mean, temp_std, pco2_mean, pco2_std,
        device='cuda'
    )
    
    isotherms[molecule_names[i]] = {
        'pco2': pco2_range,
        'alpha': preds,
        'smiles': smiles
    }

# Plot the isotherms
plt.figure(figsize=(5, 6))
colors = ['blue', 'red', 'green', 'orange']
linestyles = ['-', '-', '--', ':']

for i, (name, data) in enumerate(isotherms.items()):
    plt.semilogx(data['pco2'], data['alpha'], 
                 color=colors[i], linestyle=linestyles[i], 
                 linewidth=2, markersize=4,
                 label=f"{name} : {data['smiles']}")

plt.xlabel('CO$_{2}$ Partial Pressure (kPa)', fontsize=fontsize)
plt.ylabel(r'CO$_{\mathrm{2}}$ Loading (mol$_{\mathrm{CO2}}$/mol$_{\mathrm{A}}$)', fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.xlim(1, 1000)
#plt.ylim(0, 1.5)
plt.xscale('log')
from matplotlib.ticker import MultipleLocator
plt.gca().yaxis.set_major_locator(MultipleLocator(0.2))
plt.gca().yaxis.set_minor_locator(MultipleLocator(0.25))
plt.minorticks_on()

plt.grid(True, which='both', linewidth=0.5)
plt.legend(
    bbox_to_anchor=(0, 1.02, 1, 0.2),
    loc='lower left',
    mode='expand',
    frameon=False,
    fontsize=fontsize-7,
    ncol=2,
    markerscale=1,
    labelspacing=1,
    borderpad=0.5,
    columnspacing=0.5,
)

plt.tight_layout()
plt.show()

# Print some key values for comparison
print("\nComparison at pCO2 = 15 kPa:")
for name, data in isotherms.items():
    # Find closest value to 15 kPa
    idx = np.argmin(np.abs(data['pco2'] - 15))
    print(f"{name}: α = {data['alpha'][idx]:.3f} mol/mol")

In [ ]:
# MEA stress-test
import numpy as np
import matplotlib.pyplot as plt
fontsize = 16
# Set Times New Roman font for all text
plt.rcParams['font.family'] = 'Times New Roman'
# Define the molecules and conditions
smiles_list = ['C(CN)O' , 'C(CN)CO','C(CCN)CO','C(CCN)CCO']
molecule_names = ['MEA', '3A1P', '4A1B', '5A1P']
conc = 2.5
temp = 313.15

# Generate pCO2 range (1 to 1000 kPa, log scale)
pco2_range = np.logspace(0, 3, 20)  # 20 points from 10^0 to 10^3

# Store results
isotherms = {}

for i, smiles in enumerate(smiles_list):
    print(f"Processing {molecule_names[i]} ({smiles})...")
    
    # Create lists for this molecule across all pCO2 values
    smiles_batch = [smiles] * len(pco2_range)
    conc_batch = [conc] * len(pco2_range)
    temp_batch = [temp] * len(pco2_range)
    pco2_batch = pco2_range.tolist()
    
    # Get predictions
    preds = predict_from_smiles(
        model, smiles_batch, conc_batch, temp_batch, pco2_batch,
        conc_mean, conc_std, temp_mean, temp_std, pco2_mean, pco2_std,
        device='cuda'
    )
    
    isotherms[molecule_names[i]] = {
        'pco2': pco2_range,
        'alpha': preds,
        'smiles': smiles
    }

# Plot the isotherms
plt.figure(figsize=(5, 6))
colors = ['blue', 'red', 'green', 'orange']
linestyles = ['-', '--', '-', ':']

for i, (name, data) in enumerate(isotherms.items()):
    plt.semilogx(data['pco2'], data['alpha'], 
                 color=colors[i], linestyle=linestyles[i], 
                 linewidth=2, markersize=4,
                 label=f"{name} : {data['smiles']}")

plt.xlabel('CO$_{2}$ Partial Pressure (kPa)', fontsize=fontsize)
plt.ylabel(r'CO$_{\mathrm{2}}$ Loading (mol$_{\mathrm{CO2}}$/mol$_{\mathrm{A}}$)', fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.xlim(1, 1000)
#plt.ylim(0, 1.5)
plt.xscale('log')
from matplotlib.ticker import MultipleLocator
plt.gca().yaxis.set_major_locator(MultipleLocator(0.2))
plt.gca().yaxis.set_minor_locator(MultipleLocator(0.25))
plt.minorticks_on()

plt.grid(True, which='both', linewidth=0.5)
plt.legend(
    bbox_to_anchor=(0, 1.02, 1, 0.2),
    loc='lower left',
    mode='expand',
    frameon=False,
    fontsize=fontsize-7,
    ncol=2,
    markerscale=1,
    labelspacing=1,
    borderpad=0.5,
    columnspacing=0.5,
)

plt.tight_layout()
plt.show()

# Print some key values for comparison
print("\nComparison at pCO2 = 15 kPa:")
for name, data in isotherms.items():
    # Find closest value to 15 kPa
    idx = np.argmin(np.abs(data['pco2'] - 15))
    print(f"{name}: α = {data['alpha'][idx]:.3f} mol/mol")

# INTEGRATED GRADIENT ATTRIBUTION (ALL)

In [ ]:
unique_named_graphs = {}
for graph in combined_original_data:
    name = graph['name']
    if name not in unique_named_graphs:
        unique_named_graphs[name] = graph
unique_graph_list = list(unique_named_graphs.values())
mol_loader = DataLoader(unique_graph_list, batch_size=batch_size, shuffle=False)
# Execute the EmbeddingsExtractor
embedding_generator = EmbeddingsExtractor(model).to(device)
embedding_generator.eval()
all_embeddings = []
all_amines = []
all_data = []
with torch.no_grad():
    for data in mol_loader:
        data = data.to(device)
        combined_embedding = embedding_generator(data, 
                                                 include_conditions=False, 
                                                 extract_embeddings=True)
        all_embeddings.append(combined_embedding.cpu())
        all_amines.extend(data.name)
        all_data.extend(data.cpu().to_data_list())

# Now after the loop
all_embeddings = torch.cat(all_embeddings)
index_to_name = {i: name for i, name in enumerate(all_amines)}

In [ ]:
from captum.attr import IntegratedGradients
import torch
import numpy as np
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from matplotlib import cm, colors
import matplotlib.pyplot as plt
from PIL import Image
import io

model.eval()

# Modified forward: return pooled embedding vector (all dims)
def embedding_forward_all(x, data):
    x = embedding_generator.graph_block(x, data.edge_index, data.edge_attr)
    return x  # no indexing — return full embedding

molecule_node_attributions = {}

for molecule_name in [d.name for d in all_data]:
    print(f"\n=== Captum Analysis for {molecule_name} (ALL embeddings) ===")
    
    graph_data = next(data for data in all_data if data.name == molecule_name)
    graph_data = graph_data.to(device)
    
    x = graph_data.x.clone().detach().float().to(device)
    x.requires_grad = True
    baseline = torch.zeros_like(x)
    
    ig = IntegratedGradients(lambda x, data: embedding_forward_all(x, data).sum(dim=1))  
    # sum over embedding dims → scalar output
    
    attr = ig.attribute(
        inputs=x,
        baselines=baseline,
        additional_forward_args=(graph_data,),
        n_steps=50
    )  # shape: (num_nodes, num_node_features)
    
    # Aggregate feature-level attributions per node → scalar signed attribution
    node_signed = attr.sum(dim=1).detach().cpu().numpy()
    node_magnitude = np.abs(node_signed)
    
    molecule_node_attributions[molecule_name] = {
        'signed': node_signed,
        'magnitude': node_magnitude
    }


In [ ]:
from pathlib import Path
from rdkit import Chem
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import AllChem
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from PIL import Image, ImageChops
import io

def trim_white(img):
    """Crop extra white space around the molecule drawing."""
    bg = Image.new(img.mode, img.size, (255, 255, 255))
    diff = ImageChops.difference(img, bg)
    bbox = diff.getbbox()
    if bbox:
        return img.crop(bbox)
    return img

# === Modified IG Visualization ===
def visualize_combined_molecule(molecule_data, node_attributions, figsize=(800, 800), save_path=None, show=True):
    mol = molecule_data.mol
    if mol is None:
        print(f"No RDKit molecule available for {molecule_data.name}")
        return
    
    mol = Chem.Mol(mol)
    if mol.GetNumConformers() == 0:
        AllChem.Compute2DCoords(mol)
    
    node_attribute = node_attributions['signed']
    # Normalize the IG results to [-1, 1] range after computation
    max_abs_val = max(abs(node_attribute.min()), abs(node_attribute.max()))
    if max_abs_val > 0:
        node_attribute = node_attribute / max_abs_val  # Scale to [-1, 1]
    norm = colors.Normalize(vmin=-1, vmax=1)
    cmap = plt.colormaps.get_cmap("bwr")  # blue = negative, red = positive
    
    # Atom colors
    atom_colors = {i: cmap(norm(val))[:3] for i, val in enumerate(node_attribute)}
    
    # Bond colors: average of connected atoms
    bond_colors = {}
    for bond in mol.GetBonds():
        idx1, idx2 = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        avg_val = (node_attribute[idx1] + node_attribute[idx2]) / 2
        bond_colors[bond.GetIdx()] = cmap(norm(avg_val))[:3]
    
    plt.rcParams['font.family'] = 'Times New Roman'
    
    # Draw molecule with highlights on top
    drawer = rdMolDraw2D.MolDraw2DCairo(figsize[0], figsize[1])
    opts = drawer.drawOptions()
    opts.useBWAtomPalette()
    opts.bondLineWidth = 1.5
    opts.atomLabelFontSize = 16
    opts.highlightColour = (1.0, 1.0, 1.0)  # Make highlights more transparent
    opts.fillHighlights = True
    opts.highlightsAreCircles = False
    
    drawer.DrawMolecule(
        mol,
        highlightAtoms=list(atom_colors.keys()), 
        highlightAtomColors=atom_colors,
        highlightBonds=list(bond_colors.keys()), 
        highlightBondColors=bond_colors
    )
    drawer.FinishDrawing()
    
    png = drawer.GetDrawingText()
    image = Image.open(io.BytesIO(png)).convert("RGB")
    image = trim_white(image)
    
    if save_path:
        Path(save_path).parent.mkdir(parents=True, exist_ok=True)
    
    if show:
        fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
        im = ax.imshow(image)
        ax.axis('off')
        ax.set_title(f"{molecule_data.name} — Type: {molecule_data.type}", pad=25)
        
        # Add colorbar
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', fraction=0.046, pad=0.04)
        cbar.set_label('Signed Attribution', fontsize=12)
        # Set major ticks with interval of 0.5
        from matplotlib.ticker import MultipleLocator
        cbar.ax.xaxis.set_major_locator(MultipleLocator(0.5))
        if save_path:
            plt.savefig(f"{save_path}.png", dpi=300, bbox_inches='tight')
        plt.show()
    
    return image

# === Run visualization ===
for molecule_name in molecule_node_attributions:
    mol_data = next(d for d in all_data if d.name == molecule_name)
    mol_type = mol_data.type
    save_path = f"IntegratedGradients_fig/all/{molecule_name}_{mol_type}"
    
    visualize_combined_molecule(
        mol_data, 
        molecule_node_attributions[molecule_name], 
        save_path=save_path
    )

# INTEGRATED GRADIENT ATTRIBUTION (TRAIN)

In [ ]:
unique_named_graphs = {}
for graph in original_train_data:
    name = graph['name']
    if name not in unique_named_graphs:
        unique_named_graphs[name] = graph
unique_graph_list = list(unique_named_graphs.values())
mol_loader = DataLoader(unique_graph_list, batch_size=batch_size, shuffle=False)
# Execute the EmbeddingsExtractor
embedding_generator = EmbeddingsExtractor(model).to(device)
embedding_generator.eval()
all_embeddings = []
all_amines = []
all_data = []
with torch.no_grad():
    for data in mol_loader:
        data = data.to(device)
        combined_embedding = embedding_generator(data, 
                                                 include_conditions=False, 
                                                 extract_embeddings=True)
        all_embeddings.append(combined_embedding.cpu())
        all_amines.extend(data.name)
        all_data.extend(data.cpu().to_data_list())

# Now after the loop
all_embeddings = torch.cat(all_embeddings)
index_to_name = {i: name for i, name in enumerate(all_amines)}

In [ ]:
from captum.attr import IntegratedGradients
import torch
import numpy as np
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from matplotlib import cm, colors
import matplotlib.pyplot as plt
from PIL import Image
import io

model.eval()

# Modified forward: return pooled embedding vector (all dims)
def embedding_forward_all(x, data):
    x = embedding_generator.graph_block(x, data.edge_index, data.edge_attr)
    return x  # no indexing — return full embedding

molecule_node_attributions = {}

for molecule_name in [d.name for d in all_data]:
    print(f"\n=== Captum Analysis for {molecule_name} (ALL embeddings) ===")
    
    graph_data = next(data for data in all_data if data.name == molecule_name)
    graph_data = graph_data.to(device)
    
    x = graph_data.x.clone().detach().float().to(device)
    x.requires_grad = True
    baseline = torch.zeros_like(x)
    
    ig = IntegratedGradients(lambda x, data: embedding_forward_all(x, data).sum(dim=1))  
    # sum over embedding dims → scalar output
    
    attr = ig.attribute(
        inputs=x,
        baselines=baseline,
        additional_forward_args=(graph_data,),
        n_steps=50
    )  # shape: (num_nodes, num_node_features)
    
    # Aggregate feature-level attributions per node → scalar signed attribution
    node_signed = attr.sum(dim=1).detach().cpu().numpy()
    node_magnitude = np.abs(node_signed)
    
    molecule_node_attributions[molecule_name] = {
        'signed': node_signed,
        'magnitude': node_magnitude
    }


In [ ]:
from pathlib import Path
from rdkit import Chem
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import AllChem
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from PIL import Image, ImageChops
import io

def trim_white(img):
    """Crop extra white space around the molecule drawing."""
    bg = Image.new(img.mode, img.size, (255, 255, 255))
    diff = ImageChops.difference(img, bg)
    bbox = diff.getbbox()
    if bbox:
        return img.crop(bbox)
    return img

# === Modified IG Visualization ===
def visualize_combined_molecule(molecule_data, node_attributions, figsize=(800, 800), save_path=None, show=True):
    mol = molecule_data.mol
    if mol is None:
        print(f"No RDKit molecule available for {molecule_data.name}")
        return
    
    mol = Chem.Mol(mol)
    if mol.GetNumConformers() == 0:
        AllChem.Compute2DCoords(mol)
    
    node_attribute = node_attributions['signed']
    # Normalize the IG results to [-1, 1] range after computation
    max_abs_val = max(abs(node_attribute.min()), abs(node_attribute.max()))
    if max_abs_val > 0:
        node_attribute = node_attribute / max_abs_val  # Scale to [-1, 1]
    norm = colors.Normalize(vmin=-1, vmax=1)
    cmap = plt.colormaps.get_cmap("bwr")  # blue = negative, red = positive
    
    # Atom colors
    atom_colors = {i: cmap(norm(val))[:3] for i, val in enumerate(node_attribute)}
    
    # Bond colors: average of connected atoms
    bond_colors = {}
    for bond in mol.GetBonds():
        idx1, idx2 = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        avg_val = (node_attribute[idx1] + node_attribute[idx2]) / 2
        bond_colors[bond.GetIdx()] = cmap(norm(avg_val))[:3]
    
    plt.rcParams['font.family'] = 'Times New Roman'
    
    # Draw molecule with highlights on top
    drawer = rdMolDraw2D.MolDraw2DCairo(figsize[0], figsize[1])
    opts = drawer.drawOptions()
    opts.useBWAtomPalette()
    opts.bondLineWidth = 1.5
    opts.atomLabelFontSize = 16
    opts.highlightColour = (1.0, 1.0, 1.0)  # Make highlights more transparent
    opts.fillHighlights = True
    opts.highlightsAreCircles = False
    
    drawer.DrawMolecule(
        mol,
        highlightAtoms=list(atom_colors.keys()), 
        highlightAtomColors=atom_colors,
        highlightBonds=list(bond_colors.keys()), 
        highlightBondColors=bond_colors
    )
    drawer.FinishDrawing()
    
    png = drawer.GetDrawingText()
    image = Image.open(io.BytesIO(png)).convert("RGB")
    image = trim_white(image)
    
    if save_path:
        Path(save_path).parent.mkdir(parents=True, exist_ok=True)
    
    if show:
        fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
        im = ax.imshow(image)
        ax.axis('off')
        ax.set_title(f"{molecule_data.name} — Type: {molecule_data.type}", pad=25)
        
        # Add colorbar
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', fraction=0.046, pad=0.04)
        cbar.set_label('Signed Attribution', fontsize=12)
        # Set major ticks with interval of 0.5
        from matplotlib.ticker import MultipleLocator
        cbar.ax.xaxis.set_major_locator(MultipleLocator(0.5))
        if save_path:
            plt.savefig(f"{save_path}.png", dpi=300, bbox_inches='tight')
        plt.show()
    
    return image

# === Run visualization ===
for molecule_name in molecule_node_attributions:
    mol_data = next(d for d in all_data if d.name == molecule_name)
    mol_type = mol_data.type
    save_path = f"IntegratedGradients_fig/train/{molecule_name}_{mol_type}"
    
    visualize_combined_molecule(
        mol_data, 
        molecule_node_attributions[molecule_name], 
        save_path=save_path
    )

# INTEGRATED GRADIENT ATTRIBUTION (VAL)

In [ ]:
unique_named_graphs = {}
for graph in original_val_data:
    name = graph['name']
    if name not in unique_named_graphs:
        unique_named_graphs[name] = graph
unique_graph_list = list(unique_named_graphs.values())
mol_loader = DataLoader(unique_graph_list, batch_size=batch_size, shuffle=False)
# Execute the EmbeddingsExtractor
embedding_generator = EmbeddingsExtractor(model).to(device)
embedding_generator.eval()
all_embeddings = []
all_amines = []
all_data = []
with torch.no_grad():
    for data in mol_loader:
        data = data.to(device)
        combined_embedding = embedding_generator(data, 
                                                 include_conditions=False, 
                                                 extract_embeddings=True)
        all_embeddings.append(combined_embedding.cpu())
        all_amines.extend(data.name)
        all_data.extend(data.cpu().to_data_list())

# Now after the loop
all_embeddings = torch.cat(all_embeddings)
index_to_name = {i: name for i, name in enumerate(all_amines)}

In [ ]:
from captum.attr import IntegratedGradients
import torch
import numpy as np
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from matplotlib import cm, colors
import matplotlib.pyplot as plt
from PIL import Image
import io

model.eval()

# Modified forward: return pooled embedding vector (all dims)
def embedding_forward_all(x, data):
    x = embedding_generator.graph_block(x, data.edge_index, data.edge_attr)
    return x  # no indexing — return full embedding

molecule_node_attributions = {}

for molecule_name in [d.name for d in all_data]:
    print(f"\n=== Captum Analysis for {molecule_name} (ALL embeddings) ===")
    
    graph_data = next(data for data in all_data if data.name == molecule_name)
    graph_data = graph_data.to(device)
    
    x = graph_data.x.clone().detach().float().to(device)
    x.requires_grad = True
    baseline = torch.zeros_like(x)
    
    ig = IntegratedGradients(lambda x, data: embedding_forward_all(x, data).sum(dim=1))  
    # sum over embedding dims → scalar output
    
    attr = ig.attribute(
        inputs=x,
        baselines=baseline,
        additional_forward_args=(graph_data,),
        n_steps=50
    )  # shape: (num_nodes, num_node_features)
    
    # Aggregate feature-level attributions per node → scalar signed attribution
    node_signed = attr.sum(dim=1).detach().cpu().numpy()
    node_magnitude = np.abs(node_signed)
    
    molecule_node_attributions[molecule_name] = {
        'signed': node_signed,
        'magnitude': node_magnitude
    }


In [ ]:
from pathlib import Path
from rdkit import Chem
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import AllChem
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from PIL import Image, ImageChops
import io

def trim_white(img):
    """Crop extra white space around the molecule drawing."""
    bg = Image.new(img.mode, img.size, (255, 255, 255))
    diff = ImageChops.difference(img, bg)
    bbox = diff.getbbox()
    if bbox:
        return img.crop(bbox)
    return img

# === Modified IG Visualization ===
def visualize_combined_molecule(molecule_data, node_attributions, figsize=(800, 800), save_path=None, show=True):
    mol = molecule_data.mol
    if mol is None:
        print(f"No RDKit molecule available for {molecule_data.name}")
        return
    
    mol = Chem.Mol(mol)
    if mol.GetNumConformers() == 0:
        AllChem.Compute2DCoords(mol)
    
    node_attribute = node_attributions['signed']
    # Normalize the IG results to [-1, 1] range after computation
    max_abs_val = max(abs(node_attribute.min()), abs(node_attribute.max()))
    if max_abs_val > 0:
        node_attribute = node_attribute / max_abs_val  # Scale to [-1, 1]
    norm = colors.Normalize(vmin=-1, vmax=1)
    cmap = plt.colormaps.get_cmap("bwr")  # blue = negative, red = positive
    
    # Atom colors
    atom_colors = {i: cmap(norm(val))[:3] for i, val in enumerate(node_attribute)}
    
    # Bond colors: average of connected atoms
    bond_colors = {}
    for bond in mol.GetBonds():
        idx1, idx2 = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        avg_val = (node_attribute[idx1] + node_attribute[idx2]) / 2
        bond_colors[bond.GetIdx()] = cmap(norm(avg_val))[:3]
    
    plt.rcParams['font.family'] = 'Times New Roman'
    
    # Draw molecule with highlights on top
    drawer = rdMolDraw2D.MolDraw2DCairo(figsize[0], figsize[1])
    opts = drawer.drawOptions()
    opts.useBWAtomPalette()
    opts.bondLineWidth = 1.5
    opts.atomLabelFontSize = 16
    opts.highlightColour = (1.0, 1.0, 1.0)  # Make highlights more transparent
    opts.fillHighlights = True
    opts.highlightsAreCircles = False
    
    drawer.DrawMolecule(
        mol,
        highlightAtoms=list(atom_colors.keys()), 
        highlightAtomColors=atom_colors,
        highlightBonds=list(bond_colors.keys()), 
        highlightBondColors=bond_colors
    )
    drawer.FinishDrawing()
    
    png = drawer.GetDrawingText()
    image = Image.open(io.BytesIO(png)).convert("RGB")
    image = trim_white(image)
    
    if save_path:
        Path(save_path).parent.mkdir(parents=True, exist_ok=True)
    
    if show:
        fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
        im = ax.imshow(image)
        ax.axis('off')
        ax.set_title(f"{molecule_data.name} — Type: {molecule_data.type}", pad=25)
        
        # Add colorbar
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', fraction=0.046, pad=0.04)
        cbar.set_label('Signed Attribution', fontsize=12)
        # Set major ticks with interval of 0.5
        from matplotlib.ticker import MultipleLocator
        cbar.ax.xaxis.set_major_locator(MultipleLocator(0.5))
        if save_path:
            plt.savefig(f"{save_path}.png", dpi=300, bbox_inches='tight')
        plt.show()
    
    return image

# === Run visualization ===
for molecule_name in molecule_node_attributions:
    mol_data = next(d for d in all_data if d.name == molecule_name)
    mol_type = mol_data.type
    save_path = f"IntegratedGradients_fig/val/{molecule_name}_{mol_type}"
    
    visualize_combined_molecule(
        mol_data, 
        molecule_node_attributions[molecule_name], 
        save_path=save_path
    )

# INTEGRATED GRADIENT ATTRIBUTION (TEST)

In [ ]:
unique_named_graphs = {}
for graph in original_test_data:
    name = graph['name']
    if name not in unique_named_graphs:
        unique_named_graphs[name] = graph
unique_graph_list = list(unique_named_graphs.values())
mol_loader = DataLoader(unique_graph_list, batch_size=batch_size, shuffle=False)
# Execute the EmbeddingsExtractor
embedding_generator = EmbeddingsExtractor(model).to(device)
embedding_generator.eval()
all_embeddings = []
all_amines = []
all_data = []
with torch.no_grad():
    for data in mol_loader:
        data = data.to(device)
        combined_embedding = embedding_generator(data, 
                                                 include_conditions=False, 
                                                 extract_embeddings=True)
        all_embeddings.append(combined_embedding.cpu())
        all_amines.extend(data.name)
        all_data.extend(data.cpu().to_data_list())

# Now after the loop
all_embeddings = torch.cat(all_embeddings)
index_to_name = {i: name for i, name in enumerate(all_amines)}

In [ ]:
from captum.attr import IntegratedGradients
import torch
import numpy as np
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from matplotlib import cm, colors
import matplotlib.pyplot as plt
from PIL import Image
import io

model.eval()

# Modified forward: return pooled embedding vector (all dims)
def embedding_forward_all(x, data):
    x = embedding_generator.graph_block(x, data.edge_index, data.edge_attr)
    return x  # no indexing — return full embedding

molecule_node_attributions = {}

for molecule_name in [d.name for d in all_data]:
    print(f"\n=== Captum Analysis for {molecule_name} (ALL embeddings) ===")
    
    graph_data = next(data for data in all_data if data.name == molecule_name)
    graph_data = graph_data.to(device)
    
    x = graph_data.x.clone().detach().float().to(device)
    x.requires_grad = True
    baseline = torch.zeros_like(x)
    
    ig = IntegratedGradients(lambda x, data: embedding_forward_all(x, data).sum(dim=1))  
    # sum over embedding dims → scalar output
    
    attr = ig.attribute(
        inputs=x,
        baselines=baseline,
        additional_forward_args=(graph_data,),
        n_steps=50
    )  # shape: (num_nodes, num_node_features)
    
    # Aggregate feature-level attributions per node → scalar signed attribution
    node_signed = attr.sum(dim=1).detach().cpu().numpy()
    node_magnitude = np.abs(node_signed)
    
    molecule_node_attributions[molecule_name] = {
        'signed': node_signed,
        'magnitude': node_magnitude
    }


In [ ]:
from pathlib import Path
from rdkit import Chem
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import AllChem
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from PIL import Image, ImageChops
import io

def trim_white(img):
    """Crop extra white space around the molecule drawing."""
    bg = Image.new(img.mode, img.size, (255, 255, 255))
    diff = ImageChops.difference(img, bg)
    bbox = diff.getbbox()
    if bbox:
        return img.crop(bbox)
    return img

# === Modified IG Visualization ===
def visualize_combined_molecule(molecule_data, node_attributions, figsize=(800, 800), save_path=None, show=True):
    mol = molecule_data.mol
    if mol is None:
        print(f"No RDKit molecule available for {molecule_data.name}")
        return
    
    mol = Chem.Mol(mol)
    if mol.GetNumConformers() == 0:
        AllChem.Compute2DCoords(mol)
    
    node_attribute = node_attributions['signed']
    # Normalize the IG results to [-1, 1] range after computation
    max_abs_val = max(abs(node_attribute.min()), abs(node_attribute.max()))
    if max_abs_val > 0:
        node_attribute = node_attribute / max_abs_val  # Scale to [-1, 1]
    norm = colors.Normalize(vmin=-1, vmax=1)
    cmap = plt.colormaps.get_cmap("bwr")  # blue = negative, red = positive
    
    # Atom colors
    atom_colors = {i: cmap(norm(val))[:3] for i, val in enumerate(node_attribute)}
    
    # Bond colors: average of connected atoms
    bond_colors = {}
    for bond in mol.GetBonds():
        idx1, idx2 = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        avg_val = (node_attribute[idx1] + node_attribute[idx2]) / 2
        bond_colors[bond.GetIdx()] = cmap(norm(avg_val))[:3]
    
    plt.rcParams['font.family'] = 'Times New Roman'
    
    # Draw molecule with highlights on top
    drawer = rdMolDraw2D.MolDraw2DCairo(figsize[0], figsize[1])
    opts = drawer.drawOptions()
    opts.useBWAtomPalette()
    opts.bondLineWidth = 1.5
    opts.atomLabelFontSize = 16
    opts.highlightColour = (1.0, 1.0, 1.0)  # Make highlights more transparent
    opts.fillHighlights = True
    opts.highlightsAreCircles = False
    
    drawer.DrawMolecule(
        mol,
        highlightAtoms=list(atom_colors.keys()), 
        highlightAtomColors=atom_colors,
        highlightBonds=list(bond_colors.keys()), 
        highlightBondColors=bond_colors
    )
    drawer.FinishDrawing()
    
    png = drawer.GetDrawingText()
    image = Image.open(io.BytesIO(png)).convert("RGB")
    image = trim_white(image)
    
    if save_path:
        Path(save_path).parent.mkdir(parents=True, exist_ok=True)
    
    if show:
        fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
        im = ax.imshow(image)
        ax.axis('off')
        ax.set_title(f"{molecule_data.name} — Type: {molecule_data.type}", pad=25)
        
        # Add colorbar
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', fraction=0.046, pad=0.04)
        cbar.set_label('Signed Attribution', fontsize=12)
        # Set major ticks with interval of 0.5
        from matplotlib.ticker import MultipleLocator
        cbar.ax.xaxis.set_major_locator(MultipleLocator(0.5))
        if save_path:
            plt.savefig(f"{save_path}.png", dpi=300, bbox_inches='tight')
        plt.show()
    
    return image

# === Run visualization ===
for molecule_name in molecule_node_attributions:
    mol_data = next(d for d in all_data if d.name == molecule_name)
    mol_type = mol_data.type
    save_path = f"IntegratedGradients_fig/test/{molecule_name}_{mol_type}"
    
    visualize_combined_molecule(
        mol_data, 
        molecule_node_attributions[molecule_name], 
        save_path=save_path
    )

# INTEGRATED GRADIENT ATTRIBUTION (MADE UP MOLECULE)

In [ ]:
# MADE UP DATA
def canonicalize_smiles(smiles):
    """Convert SMILES to canonical form for consistent comparison"""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        return Chem.MolToSmiles(mol, canonical=True)
    except:
        return None

def create_existing_amine_lookup(unique_graph_list_with_embeddings):
    """Create a lookup dictionary from canonical SMILES to existing amine data"""
    smiles_to_data = {}
    
    for data_point in unique_graph_list_with_embeddings:
        canonical_smiles = None
        
        # Get canonical SMILES from the smiles attribute
        if hasattr(data_point, 'smiles') and data_point.smiles:
            canonical_smiles = canonicalize_smiles(data_point.smiles)
        
        # Fallback to mol attribute if smiles didn't work
        elif hasattr(data_point, 'mol') and data_point.mol is not None:
            try:
                canonical_smiles = Chem.MolToSmiles(data_point.mol, canonical=True)
            except:
                continue
        
        if canonical_smiles and canonical_smiles not in smiles_to_data:
            smiles_to_data[canonical_smiles] = data_point
    
    return smiles_to_data

def create_clean_graph_from_existing(existing_data):
    """Create a clean graph from existing data, removing experimental conditions"""
    # Create a new graph with just the essential molecular information
    clean_graph = smiles_to_graph(existing_data.smiles, None)
    clean_graph.name = existing_data.name  # Keep the original name
    
    # Keep only the molecular structure attributes, not experimental conditions
    if hasattr(existing_data, 'type'):
        clean_graph.type = existing_data.type
    if hasattr(existing_data, 'primary_count'):
        clean_graph.primary_count = existing_data.primary_count
    if hasattr(existing_data, 'secondary_count'):
        clean_graph.secondary_count = existing_data.secondary_count
    if hasattr(existing_data, 'tertiary_count'):
        clean_graph.tertiary_count = existing_data.tertiary_count
    # Explicitly NOT copying: conc, temp, pco2, aco2, ref, DOI, embedding
    
    return clean_graph

# Define SMILES data with names
smiles_data = [
    {'name': 'M0', 'smiles': 'C(CNCCO)N'},
    {'name': 'M1', 'smiles': 'C(CCNCO)N'},
    {'name': 'M2', 'smiles': 'C(NCCCO)N'},
    {'name': 'M3', 'smiles': 'C(COCCN)N'}, 
]

# Create lookup for existing amines from your collection
existing_amine_lookup = create_existing_amine_lookup(unique_graph_list_with_embeddings)

# Process all compounds - use existing molecular data but create clean versions
made_up_molecule = []
found_existing = []
created_new = []

print("Processing input compounds...")
for item in smiles_data:
    canonical_smiles = canonicalize_smiles(item['smiles'])
    
    if canonical_smiles is None:
        print(f"Warning: Could not parse SMILES '{item['smiles']}' for {item['name']}")
        continue
    
    # Check if molecule already exists in your amine collection
    if canonical_smiles in existing_amine_lookup:
        # Use existing molecular data but create clean version without experimental conditions
        existing_amine = existing_amine_lookup[canonical_smiles]
        clean_graph = create_clean_graph_from_existing(existing_amine)
        made_up_molecule.append(clean_graph)
        found_existing.append((item['name'], existing_amine.name, canonical_smiles))
        print(f"[!] Using existing compound: {existing_amine.name} (you requested as '{item['name']}') - cleaned format")
        
    else:
        # Create new graph from SMILES - this is a new compound
        print(f"[OK] Creating new compound: {item['name']} ({canonical_smiles})")
        graph = smiles_to_graph(item['smiles'], None)
        graph.name = item['name']
        made_up_molecule.append(graph)
        created_new.append((item['name'], canonical_smiles))

# Print summary
print(f"\n=== PROCESSING SUMMARY ===")
print(f"Input compounds: {len(smiles_data)}")
print(f"Found existing (cleaned): {len(found_existing)}")
print(f"Created new: {len(created_new)}")

if found_existing:
    print("\nExisting compounds (cleaned format):")
    for input_name, original_name, smiles in found_existing:
        print(f"  {input_name} <- based on existing {original_name}")

if created_new:
    print("\nNew compounds created:")
    for name, smiles in created_new:
        print(f"  {name} -> {smiles}")

print(f"\n=== GENERATING EMBEDDINGS FOR ALL {len(made_up_molecule)} COMPOUNDS ===")

# Remove duplicates by name
unique_named_graphs = {}
for graph in made_up_molecule:
    name = graph.name
    if name not in unique_named_graphs:
        unique_named_graphs[name] = graph

unique_graph_list = list(unique_named_graphs.values())
mol_loader = DataLoader(unique_graph_list, batch_size=batch_size, shuffle=False)

# Execute the EmbeddingsExtractor
embedding_generator = EmbeddingsExtractor(model).to(device)
embedding_generator.eval()
all_embeddings = []
all_amines = []
all_data = []

with torch.no_grad():
    for data in mol_loader:
        data = data.to(device)
        combined_embedding = embedding_generator(data, 
                                                 include_conditions=False, 
                                                 extract_embeddings=True)
        all_embeddings.append(combined_embedding.cpu())
        all_amines.extend(data.name)
        all_data.extend(data.cpu().to_data_list())

# Combine all embeddings
all_embeddings = torch.cat(all_embeddings)
index_to_name = {i: name for i, name in enumerate(all_amines)}

print(f"\n=== FINAL RESULTS ===")
print(f"Total compounds processed: {len(unique_graph_list)}")
print(f"Total embeddings generated: {all_embeddings.shape[0]}")
print(f"Embedding shape: {all_embeddings.shape}")

print("\nAll compound names and indices:")
for i, name in index_to_name.items():
    print(f"  Index {i}: {name}")

In [ ]:
from captum.attr import IntegratedGradients
import torch
import numpy as np
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from matplotlib import cm, colors
import matplotlib.pyplot as plt
from PIL import Image
import io

model.eval()

# Modified forward: return pooled embedding vector (all dims)
def embedding_forward_all(x, data):
    x = embedding_generator.graph_block(x, data.edge_index, data.edge_attr)
    return x  # no indexing — return full embedding

molecule_node_attributions = {}

for molecule_name in [d.name for d in all_data]:
    print(f"\n=== Captum Analysis for {molecule_name} (ALL embeddings) ===")
    
    graph_data = next(data for data in all_data if data.name == molecule_name)
    graph_data = graph_data.to(device)
    
    x = graph_data.x.clone().detach().float().to(device)
    x.requires_grad = True
    baseline = torch.zeros_like(x)
    
    ig = IntegratedGradients(lambda x, data: embedding_forward_all(x, data).sum(dim=1))  
    # sum over embedding dims → scalar output
    
    attr = ig.attribute(
        inputs=x,
        baselines=baseline,
        additional_forward_args=(graph_data,),
        n_steps=50
    )  # shape: (num_nodes, num_node_features)
    
    # Aggregate feature-level attributions per node → scalar signed attribution
    node_signed = attr.sum(dim=1).detach().cpu().numpy()
    node_magnitude = np.abs(node_signed)
    
    molecule_node_attributions[molecule_name] = {
        'signed': node_signed,
        'magnitude': node_magnitude
    }


In [ ]:
from pathlib import Path
from rdkit import Chem
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import AllChem
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from PIL import Image, ImageChops
import io

def trim_white(img):
    """Crop extra white space around the molecule drawing."""
    bg = Image.new(img.mode, img.size, (255, 255, 255))
    diff = ImageChops.difference(img, bg)
    bbox = diff.getbbox()
    if bbox:
        return img.crop(bbox)
    return img

# === Modified IG Visualization ===
def visualize_combined_molecule(molecule_data, node_attributions, figsize=(800, 800), save_path=None, show=True):
    mol = molecule_data.mol
    if mol is None:
        print(f"No RDKit molecule available for {molecule_data.name}")
        return
    
    mol = Chem.Mol(mol)
    if mol.GetNumConformers() == 0:
        AllChem.Compute2DCoords(mol)
    
    node_attribute = node_attributions['signed']
    # Normalize the IG results to [-1, 1] range after computation
    max_abs_val = max(abs(node_attribute.min()), abs(node_attribute.max()))
    if max_abs_val > 0:
        node_attribute = node_attribute / max_abs_val  # Scale to [-1, 1]
    norm = colors.Normalize(vmin=-1, vmax=1)
    cmap = plt.colormaps.get_cmap("bwr")  # blue = negative, red = positive
    
    # Atom colors
    atom_colors = {i: cmap(norm(val))[:3] for i, val in enumerate(node_attribute)}
    
    # Bond colors: average of connected atoms
    bond_colors = {}
    for bond in mol.GetBonds():
        idx1, idx2 = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        avg_val = (node_attribute[idx1] + node_attribute[idx2]) / 2
        bond_colors[bond.GetIdx()] = cmap(norm(avg_val))[:3]
    
    plt.rcParams['font.family'] = 'Times New Roman'
    
    # Draw molecule with highlights on top
    drawer = rdMolDraw2D.MolDraw2DCairo(figsize[0], figsize[1])
    opts = drawer.drawOptions()
    opts.useBWAtomPalette()
    opts.bondLineWidth = 1.5
    opts.atomLabelFontSize = 16
    opts.highlightColour = (1.0, 1.0, 1.0)  # Make highlights more transparent
    opts.fillHighlights = True
    opts.highlightsAreCircles = False
    
    drawer.DrawMolecule(
        mol,
        highlightAtoms=list(atom_colors.keys()), 
        highlightAtomColors=atom_colors,
        highlightBonds=list(bond_colors.keys()), 
        highlightBondColors=bond_colors
    )
    drawer.FinishDrawing()
    
    png = drawer.GetDrawingText()
    image = Image.open(io.BytesIO(png)).convert("RGB")
    image = trim_white(image)
    
    if save_path:
        Path(save_path).parent.mkdir(parents=True, exist_ok=True)
    
    if show:
        fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
        im = ax.imshow(image)
        ax.axis('off')
        ax.set_title(f"{molecule_data.name} — Type: {molecule_data.type}", pad=25)
        
        # Add colorbar
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', fraction=0.046, pad=0.04)
        cbar.set_label('Signed Attribution', fontsize=12)
        # Set major ticks with interval of 0.5
        from matplotlib.ticker import MultipleLocator
        cbar.ax.xaxis.set_major_locator(MultipleLocator(0.5))
        if save_path:
            plt.savefig(f"{save_path}.png", dpi=300, bbox_inches='tight')
        plt.show()
    
    return image

# === Run visualization ===
for molecule_name in molecule_node_attributions:
    mol_data = next(d for d in all_data if d.name == molecule_name)
    mol_type = mol_data.type
    save_path = f"IntegratedGradients_fig/made_up/{molecule_name}_{mol_type}"
    
    visualize_combined_molecule(
        mol_data, 
        molecule_node_attributions[molecule_name], 
        save_path=save_path
    )

# BACKUP

In [ ]:
""" import numpy as np
if not hasattr(np, "bool"):
    np.bool = np.bool_
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Stack all embeddings into a 2D array (num_molecules x 64)
embeddings = np.stack([d.embedding if isinstance(d.embedding, np.ndarray) else d.embedding.numpy()
                       for d in unique_graph_list_with_embeddings])

# Step 2: Compute correlation matrix (64 x 64)
corr_matrix = np.corrcoef(embeddings, rowvar=False)

# Step 3: Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, cmap='coolwarm', center=0, square=True)
plt.xlabel("Embedding Dimension")
plt.ylabel("Embedding Dimension")
plt.show()
# Extract off-diagonal correlations
off_diag_corrs = corr_matrix[np.triu_indices_from(corr_matrix, k=1)]

# Test against null hypothesis of orthogonality (mean ≈ 0)
from scipy import stats
t_stat, p_value = stats.ttest_1samp(off_diag_corrs, 0)

# Also check distribution statistics
print(f"Mean absolute correlation: {np.mean(np.abs(off_diag_corrs)):.3f}")
print(f"Std of correlations: {np.std(off_diag_corrs):.3f}") """

In [ ]:
""" # Quick significance test you can run:
from scipy import stats

# Test if mean significantly different from orthogonal (0)
t_stat, p_value = stats.ttest_1samp(np.abs(off_diag_corrs), 0)
print(f"P-value for non-orthogonality: {p_value}")

# Also check what fraction are "strong" correlations
strong_corr_fraction = np.mean(np.abs(off_diag_corrs) > 0.3)
print(f"Fraction with |r| > 0.3: {strong_corr_fraction:.2%}") """